In [19]:
import os
import re
import json

import dotenv
import pymongo
from bson import ObjectId

In [2]:
def serialize_obj(obj):
    """Recursively convert ObjectId to str and prepare for JSON serialization."""
    if isinstance(obj, ObjectId):
        return str(obj)
    if isinstance(obj, list):
        return [serialize_obj(item) for item in obj]
    if isinstance(obj, dict):
        return {key: serialize_obj(value) for key, value in obj.items()}
    return obj


def connect_to_db(collection):
    dotenv.load_dotenv()
    mongodb_uri = os.getenv("MONGODB_URI")
    DBclient = pymongo.MongoClient(mongodb_uri)
    db = DBclient["ML2-Abschlussprojekt"]
    return db[collection]

In [3]:
def get_all_assessments(semester_id):
    results = []
    collection = connect_to_db("Assessment_Data")
    cursor = collection.find({
        "semester_id":ObjectId(semester_id),
    })
    for document in cursor:
          results.append(document)
    return serialize_obj(results)

In [4]:
def anonymise(student, text):
    if student["gender"].lower()=="männlich":
        aliasname = "[John]"
    else:
        aliasname = "[Jane]"
    alias_text = text.replace(student["firstname"], aliasname)
    return alias_text

In [5]:
assessments = get_all_assessments("665d8ff86ce65f51c7e17e34")

In [36]:
def collect_notes(assessments):
    """Collects notes from criteria, if there is more than one note."""
    notes = dict()
    for assessment in assessments:
        for k, v in assessment["allgemeines_lernen"].items():
            if v["notes"]:
                notes.setdefault(k, []).append(v["notes"])
    notes = {k:v for k,v in notes.items() if len(v)>1}
    return json.dumps(notes, ensure_ascii=False)

In [37]:
notes = collect_notes(assessments)

In [38]:
notes

'{"SchulinhalteAbrufen": ["manchmal hat er Schwierigkeiten, bei Prüfungen zu zeigen, was er kann.", "ab und zu fällt es Bruce schwer, bei Prüfungen zu zeigen, was er kann."], "AktivTeilnehmen": ["Bruce hat viele schulische Interessen. Insbesondere in Physik und Chemie ist er sehr gut", "insbesondere bei Fächer, die ihn interessieren"]}'

In [39]:
import openai

def get_api_key():
    dotenv.load_dotenv()
    os.getenv("OPENAI_APIKEY")
    return os.getenv("OPENAI_APIKEY")
    

def prompt_for_note_recommendation(notes):
    system_content = f"""
    Du wirst ein JSON-Object bekommen, das aus einem Key-Value-Pair besteht.
    Die Values sind mehrere Strings, die mit einem Komma getrennt wurden.
    Dein Task ist es, die Strings so zusammenzusetzen, dass sie einen Fliesstext ergeben.
    Der Fliesstext soll professionell sein.
    Der Fliesstext soll nur Informationen enthalten, die in den Values mitgegeben werden.
    Der Fliesstext soll die Schweizer Rechtschreibung nutzen, das heisst, Scharf-S wird durch Doppel-S ersetzt.
    Die Antwort soll ein JSON-Object sein.
    Der Key soll dabei unverändert bleiben und die Values sollen mit dem Fliesstext ersetzt werden.
    """
    first_prompt = f"""
    Bitte mache aus den Values des folgenden Dictionary einen professionellen Fliesstext:
    {notes}
    """
    return [
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": """
    'AktivTeilnehmen': {'insbesondere bei Fächer, die ihn interessieren', 'insbesondere in Physik und Chemie'}
    'SchulinhalteAbrufen': {'sehr Themenabhängig', 'manchmal, manchmal nicht', 'grundsätzlich schon, aber nur, wenn es ihn interessiert'}
    'AufmerksamSein': {'wenn er einen schlechten Tag hat, geht es nicht', 'meistens gelingt es gut, ausser er hat private Probleme'}
    'SchulinhalteMerken': {'wenn ihn das Thema interessiert', 'nur bei speziellen Themen', 'an guten Tagen', 'nicht unbedingt schulische Themen, aber sonst kann er sich viele Sachen merken'}
    'LeistungZeigen': {'häufig hat er Migräne', 'wenn er zu wenig geschlafen hat, ist es schwierig'}
            """
        },
        {
            "role": "assistant",
            "content": """
    'AktivTeilnehmen': {'Insbesondere bei Fächer, die ihn interessieren, wie beispielsweise Physik oder Chemie.'}
    'SchulinhalteAbrufen': {'Grundsätzlich gelingt es ihm, wenn ihn ein Thema interessiert. Manchmal ist es aber auch schwierig für ihn.'}
    'AufmerksamSein': {'Meistens gelingt es ihm, wenn er aber einen schlechten Tag oder Probleme in seinem privaten Umfeld hat, fällt es ihm schwer.'}
    'SchulinhalteMerken': {'Er hat grundsätzlich ein gutes Gedächtnis, aber nicht bei allen schulischen Themen, kann er sich die Inhalte merken. An guten Tagen geht es besser.'}
    'LeistungZeigen': {'wenn er Migräne hat oder zu wenig geschlafen hat, ist es schwierig. Leider kommt das häufiger vor.'}
            """
        },
        {
            "role": "user",
            "content": first_prompt
        },
    ]


def _openai(messages, chain=True) -> openai.types.chat.chat_completion.Choice:
    """
    Request is sent to OpenAI
    Return the response in a ChatCompletion-Object
    """
    model_result = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2,
        top_p=1,
    )
    response = model_result.choices[0]
    if chain:
        chain = messages + [response.message]
        return response.message.content, chain
    return response.message.content



def get_recommendation_notes(assessment):
    notes = collect_notes(assessment)
    messages = prompt_for_note_recommendation(notes)
    openai.api_key = get_api_key()
    answer = _openai(messages, False)
    return answer

In [40]:
answer = get_recommendation_notes(assessments)
answer

Checkpoint


'\n{\n    "SchulinhalteAbrufen": ["Manchmal hat Bruce Schwierigkeiten, bei Prüfungen sein volles Potenzial zu zeigen. Ab und zu fällt es ihm schwer, sein Wissen in Prüfungssituationen abzurufen."],\n    "AktivTeilnehmen": ["Bruce zeigt grosses Interesse an verschiedenen schulischen Themen. Besonders in den Fächern Physik und Chemie zeigt er herausragende Leistungen."]\n}'

In [44]:
json.loads(answer)

{'SchulinhalteAbrufen': ['Manchmal hat Bruce Schwierigkeiten, bei Prüfungen sein volles Potenzial zu zeigen. Ab und zu fällt es ihm schwer, sein Wissen in Prüfungssituationen abzurufen.'],
 'AktivTeilnehmen': ['Bruce zeigt grosses Interesse an verschiedenen schulischen Themen. Besonders in den Fächern Physik und Chemie zeigt er herausragende Leistungen.']}